# HMM Continuous Timeseries Dataset

This synthetic dataset is generated using a Hidden Markov Model (HMM) with continuous valued observations. Each sample in the dataset is a multivariate timeseries with a variable length. The dataset includes an identifier for each sample, a set of continuous observations for each timestep, and a corresponding state label for each timestep.

## Dataset Generation Process

1. **HMM Parameters**:
   - **Number of States (M)**: The HMM has a specified number of states.
   - **Number of Gaussians (K)**: Each state is modeled using a Gaussian Mixture Model (GMM) with a specified number of Gaussians.
   - **Dimensions of Observations (D)**: Each observation at a timestep is a D-dimensional vector.

2. **State Transition Matrix (A)**:
   - The transition probabilities between states are defined by a matrix where the probability of staying in the same state is randomly chosen between a minimum and maximum value for each state.
   - The remaining probability is evenly distributed among transitions to other states.

3. **Initial State Distribution (pi)**:
   - The initial state distribution is uniform, where each state has an equal probability.

4. **GMM Parameters**:
   - **Mixture Proportions (R)**: The proportions for each Gaussian in the GMM are drawn from a Dirichlet distribution.
   - **Means (mu)**: The means of the Gaussians are randomly initialized.
   - **Covariances (sigma)**: The covariances of the Gaussians are diagonal matrices with random values.

5. **Observation Generation**:
   - States for each timestep are generated according to the HMM's transition probabilities.
   - Observations are generated from the GMM corresponding to the current state.

## Dataset Structure

- **Sample ID**: A unique identifier for each sample.
- **Variable Length Timeseries**: Each sample is a timeseries with a length randomly chosen between specified minimum and maximum values.
- **Observations**: Each observation is a D-dimensional vector of continuous values.
- **State Labels**: Each timestep in the timeseries has a corresponding state label.


## Summary

The HMM Continuous Timeseries Dataset presents a challenging task for timeseries annotation algorithms, where the goal is to accurately classify each timestep within the sequences based on the hidden states of the HMM. The variable length of the samples adds complexity, making it an excellent dataset for advanced timeseries analysis techniques.

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
dataset_name = "hmm_continuous"
test_size = 0.2

In [3]:
output_dir = f'./../../processed/{dataset_name}/'
outp_fname = os.path.join(output_dir, f'{dataset_name}.csv')
train_outp_fname = os.path.join(output_dir, f'{dataset_name}_train.csv')
test_outp_fname = os.path.join(output_dir, f'{dataset_name}_test.csv')
test_key_outp_fname = os.path.join(output_dir, f'{dataset_name}_test_key.csv')

In [4]:

def generate_hmm_parameters(M: int, K: int, D: int, min_stay_prob: float, max_stay_prob: float):
    """
    Generate Hidden Markov Model (HMM) parameters.

    Args:
        M (int): Number of states.
        K (int): Number of Gaussians in the Gaussian Mixture Model (GMM).
        D (int): Number of dimensions in the observation.
        min_stay_prob (float): Minimum probability of staying in the same state.
        max_stay_prob (float): Maximum probability of staying in the same state.

    Returns:
        pi (np.ndarray): Initial state distribution. Shape is (M,).
        A (np.ndarray): State transition matrix. Shape is (M, M).
        R (np.ndarray): Mixture of proportions. Shape is (M, K).
        mu (np.ndarray): Means of the Gaussians. Shape is (M, K, D).
        sigma (np.ndarray): Covariances of the Gaussians. Shape is (M, K, D, D).
    """
    pi = np.ones(M) / M # initial state distribution (uniform)
    
    A = np.zeros((M, M))
    for i in range(M):
        stay_prob = np.random.uniform(min_stay_prob, max_stay_prob)
        A[i, :] += (1 - stay_prob) / (M - 1)
        A[i, i] = stay_prob # overwrite diagonals

    
    
    R = np.random.dirichlet(np.ones(K), size=M)
    
    mu = np.random.randn(M, K, D)
    
    sigma = np.zeros((M, K, D, D))
    for m in range(M):
        for k in range(K):
            sigma[m, k, :, :] = np.diag(np.random.rand(D))
    return pi, A, R, mu, sigma

In [5]:
def generate_hmm_samples(
    N: int,
    T: int,
    pi: np.ndarray,
    A: np.ndarray,
    R: np.ndarray,
    mu: np.ndarray,
    sigma: np.ndarray,
):
    """
    Generate dataset from the Hidden Markov Model (HMM).

    Args:
        N (int): Number of samples to generate.
        T (int): Length of each sample.
        pi (np.ndarray): Initial state distribution. Shape is (M,).
        A (np.ndarray): State transition matrix. Shape is (M, M).
        R (np.ndarray): Mixture of proportions. Shape is (M, K).
        mu (np.ndarray): Means of the Gaussians. Shape is (M, K, D).
        sigma (np.ndarray): Covariances of the Gaussians. Shape is (M, K, D, D).
        
        where
            M: number of states
            K: number of Gaussians in the GMM
            D: number of dimensions in the observation

    Returns:
        observations (np.ndarray): Generated observations. Shape is (N, T, D).
        labels (np.ndarray): State labels for each timestep. Shape is (N, T).
    """
    M, K, D = mu.shape
    observations = np.zeros((N, T, D))
    labels = np.zeros((N, T), dtype=int)
    
    for n in range(N):
        states = np.zeros(T, dtype=int)
        obs = np.zeros((T, D))
        # starting state
        states[0] = np.random.choice(M, p=pi)
        
        # rest of the states over the sequence
        for t in range(1, T):
            states[t] = np.random.choice(M, p=A[states[t-1]])
        
        # observations over the sequence
        for t in range(T):
            state = states[t]
            component = np.random.choice(K, p=R[state])
            obs[t] = np.random.multivariate_normal(mu[state, component], sigma[state, component])
        
        observations[n] = obs
        labels[n] = states
        
        if n % 100 == 0 and n > 0:
            print(f"Generated {n} samples...")
    
    print("Done generating samples.")
    return observations, labels

In [6]:
def adjust_variable_length(
        observations: np.ndarray,
        labels: np.ndarray,
        min_len: int,
        max_len: int
    ):
    """
    Adjust samples and labels to have variable lengths.

    Args:
        observations (np.ndarray): Generated observations. Shape is (N, T, D).
        labels (np.ndarray): State labels for each timestep. Shape is (N, T).
        min_len (int): Minimum length of the samples.
        max_len (int): Maximum length of the samples.

    Returns:
        var_len_observations (list of np.ndarray): List of observations with variable lengths.
        var_len_labels (list of np.ndarray): List of labels with variable lengths.
    """
    N, T, D = observations.shape
    var_len_observations = []
    var_len_labels = []

    for i in range(N):
        length = np.random.randint(min_len, max_len + 1)
        if length > T:
            raise ValueError(
                f"Maximum length {max_len} cannot be greater than the original length {T}"
            )
        start_idx = np.random.randint(0, T - length + 1)
        var_len_observations.append(observations[i, start_idx:start_idx + length, :])
        var_len_labels.append(labels[i, start_idx:start_idx + length])
    
    return var_len_observations, var_len_labels

In [7]:
def convert_to_dataframe(observations: list, labels: list) -> pd.DataFrame:
    """
    Convert the variable length observations and labels to a pandas DataFrame.

    Args:
        observations (list of np.ndarray): List of observations with variable lengths.
        labels (list of np.ndarray): List of labels with variable lengths.

    Returns:
        pd.DataFrame: DataFrame containing the data with columns for sample_id, label,
                      and observation dimensions.
    """
    data = []
    for i, (obs, lbl) in enumerate(zip(observations, labels)):
        for obs_t, lbl_t in zip(obs, lbl):
            row = {"sample_id": i, "label": int(lbl_t)}
            row.update({
                f"observation_dim_{d+1}": float(obs_t[d]) for d in range(obs.shape[1])
            })
            data.append(row)
    
    df = pd.DataFrame(data)
    return df

In [8]:
def generate_hmm_dataset(
        N: int,
        min_seq_len: int,
        max_seq_len: int,
        M: int,
        K: int,
        D: int,
        min_stay_prob: float,
        max_stay_prob: float
    ) -> pd.DataFrame:
    """
    Generate HMM dataset with variable length sequences and return as pandas DataFrame.

    Args:
        N (int): Number of samples to generate.
        min_seq_len (int): Minimum length of the sequences. Must be >= 10
        max_seq_len (int): Maximum length of the sequences.
        M (int): Number of states.
        K (int): Number of Gaussians in the Gaussian Mixture Model (GMM).
        D (int): Number of dimensions in the observation.
        min_stay_prob (float): Minimum probability of staying in the same state.
        max_stay_prob (float): Maximum probability of staying in the same state.

    Returns:
        pd.DataFrame: DataFrame containing the generated dataset.
    """
    pi, A, R, mu, sigma = generate_hmm_parameters(M, K, D, min_stay_prob, max_stay_prob)
    assert min_seq_len >= 10, "Minimum sequence length should be >= 10"
    assert min_seq_len <= max_seq_len, "Minimum sequence length cannot be greater than maximum sequence length"
    observations, labels = generate_hmm_samples(
        N=N,
        T=max_seq_len,
        pi=pi,
        A=A,
        R=R,
        mu=mu,
        sigma=sigma,
    )
    var_len_observations, var_len_labels = adjust_variable_length(observations, labels, min_seq_len, max_seq_len)
    df = convert_to_dataframe(var_len_observations, var_len_labels)
    return df

In [9]:
def set_seeds(seed=888):
    np.random.seed(seed)

In [10]:
N = 300
min_seq_len = 50
max_seq_len = N
# number of states
M = 4
# number of Gaussians
K = 5
# dimensionality of data
D = 3
min_stay_prob = 0.8
max_stay_prob = 0.99

set_seeds(1)
data = generate_hmm_dataset(
    N=N,
    min_seq_len=min_seq_len,
    max_seq_len=max_seq_len,
    M=M,
    K=K,
    D=D,
    min_stay_prob=min_stay_prob,
    max_stay_prob=max_stay_prob,
)


Generated 100 samples...
Generated 200 samples...
Done generating samples.


In [11]:
data.shape

(51534, 5)

In [12]:
data.head()

,sample_id,label,observation_dim_1,observation_dim_2,observation_dim_3
0,0,1,0.323794,-1.134339,-0.988766
1,0,1,-0.673589,0.291930,-1.163936
2,0,1,0.141649,2.867647,1.001803
3,0,1,0.100120,0.096912,0.706180
4,0,1,0.536475,2.487519,0.107215


## Adding time column

In [13]:
grouped = data.groupby("sample_id")
with_time = []
for i, group in grouped:
    group['timestep'] = list(range(len(group)))
    with_time.append(group)

data = pd.concat(with_time)
data.head()

,sample_id,label,observation_dim_1,observation_dim_2,observation_dim_3,timestep
0,0,1,0.323794,-1.134339,-0.988766,0
1,0,1,-0.673589,0.291930,-1.163936,1
2,0,1,0.141649,2.867647,1.001803,2
3,0,1,0.100120,0.096912,0.706180,3
4,0,1,0.536475,2.487519,0.107215,4


# Save Main Data File

In [14]:
os.makedirs(output_dir, exist_ok=True)

data.to_csv(outp_fname, index=False, float_format="%.4f")

## Train/Test Split

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def split_by_sample_id(data: pd.DataFrame, test_size: float, random_state: int = None):
    """
    Split the data into train and test sets by sample_id.

    Args:
        data (pd.DataFrame): The dataset containing sample_id, label, and observation dimensions.
        test_size (float): The proportion of the dataset to include in the test split.
        random_state (int, optional): Random seed for reproducibility.

    Returns:
        train_df (pd.DataFrame): The training data.
        test_df (pd.DataFrame): The testing data.
        test_keys (pd.DataFrame): The testing keys containing sample_id and label.
    """
    # Get unique sample IDs
    unique_ids = data['sample_id'].unique()

    # Split the sample IDs into train and test sets
    train_ids, test_ids = train_test_split(unique_ids, test_size=test_size, random_state=random_state)

    # Initialize lists for storing the split data
    train_data, test_data, test_keys = [], [], []

    # Group the data by sample_id
    grouped = data.groupby('sample_id')

    # Assign each group to either train or test based on the split
    for sample_id, group in grouped:
        if sample_id in train_ids:
            train_data.append(group)
        else:
            test_key = group[['sample_id', 'timestep', 'label']].reset_index(drop=True)
            test_df = group.drop(columns=['label']).reset_index(drop=True)
            test_data.append(test_df)
            test_keys.append(test_key)

    # Concatenate the data into final DataFrames
    train_df = pd.concat(train_data).reset_index(drop=True)
    test_df = pd.concat(test_data).reset_index(drop=True)
    test_keys = pd.concat(test_keys).reset_index(drop=True)

    return train_df, test_df, test_keys

# Example usage:
# df = generate_and_save_hmm_dataset(100, 50, 100, 3, 2, 2, 0.7, 0.9, seed=42)
train_df, test_df, test_keys = split_by_sample_id(data, test_size=test_size, random_state=42)

print(data.shape, train_df.shape, test_df.shape, test_keys.shape)

(51534, 6) (40493, 6) (11041, 5) (11041, 3)


In [18]:
train_df.to_csv(train_outp_fname, index=False, float_format="%.4f")
test_df.to_csv(test_outp_fname, index=False, float_format="%.4f")
test_keys.to_csv(test_key_outp_fname, index=False, float_format="%.4f")